In [ ]:
from molsim import MolecularDynamics
import matplotlib.pyplot as plt
import numpy as np

plt.rc("font", **{"size": 16})
plt.rc("axes", **{"formatter.use_mathtext": True, "labelpad": 8})

# Molecular Dynamics techniques

### Initialize Molecular Dynamics system

The MD program consists of a main MolecularDynamics object, found in `src/molecularDynamics/md.h`. The object has a constructor, that allows you to set a bunch of different settings that you might want to use in this simulation. Furthermore you will find the thermostats in `src/molecularDynamics/thermostats.h` and some samplers in `src/molecularDynamics/samplers.h`.

### MolecularDynamics object
The `MolecularDynamics` struct has a bunch of different functions. First there is the constructor `MolecularDynamics::MolecularDynamics`, which, from the input settings, sets up the simulation. First it precomputes some properties, like volume and density. Secondly, it initializes the velocities according the Maxwell-Boltzmann distribution (note that because mass = 1, in this case and therefore momentum = velocity). 

It also initializes all positions on a cubic lattice by calling `MolecularDynamics::latticeInitialization`, which prevents overlaps from happening, but a further minimization to the closest local minimum is done by `MolecularDynamics::gradientDescent`.

### Force calculation and integration
In MD one propagates the system by integrating the equations of motion. The integrator used for the equations of motion is the velocity Verlet integrator, which is a symplectic integrator. This means that the Hamiltonian is conserved, or, in other words, the drift or error in the ideally constant Hamiltonian is bounded. The Hamiltonian for an NVE system is (settings $m=1$)

$\mathcal{H}(p, q) = \sum_i^{N} \frac{p_i^2}{2} + \mathcal{U}(q)$

yielding the equations of motion:

$\dot{q} = \frac{\partial \mathcal{H}}{\partial p} = \frac{p}{m}$

$\dot{p} = -\frac{\partial \mathcal{H}}{\partial q} = -\frac{\partial \mathcal{U}}{\partial q}$.

In the velocity verlet algorithm these equations of motion are governed by updating the velocity in two half steps, once with the forces at time $t$ and once with the forces at time $t+\Delta t$.

This leads to integration scheme:

$v(t + \frac{\Delta t}{2}) = v(t) + F(t) \frac{\Delta t}{2}$

$q(t + \Delta t) = q(t) + v(t) \Delta t + F(t) \frac{(\Delta t)^2}{2}$

$v(t + \Delta t) = v(t + \frac{\Delta t}{2}) + F(t + \Delta t) \frac{\Delta t}{2}$

where

$F(t) = -\frac{\partial \mathcal{U}(q(t))}{\partial q}$.

In the code this force term is calculated in the method `MolecularDynamics::calculateForce`. This is the most computationally demanding function as it is a loop over all particle pairs. It can be nicely combined with computing the potential energy and the virial, which is used in the pressure. All particles have interaction with eachother according to the so-called Lennard-Jones potential - which has a repulsive term $r^{-12}$, which can be seen as the Pauli repulsion and an attraction term $r^{-6}$, which is often called the van der Waals term, as the van der Waals interaction also diminishes with $r^{-6}$. 

$\mathcal{U}(q) = \frac{1}{2} \sum_i^N \sum_{j \neq i}^N U_{LJ}(r_{ij})$

where

$r_{ij} = |q_i - q_j|$

and 

$U_{LJ}(r_{ij}) = 4 \epsilon \left((\frac{\sigma}{r})^{12} - (\frac{\sigma}{r})^{6} \right)$.

This force is then used in `MolecularDynamics::integrate` to sequentially update the momenta, positions and forces in given the steps in the equations of motion. At the same time, it might also make calls to the thermostat, which allows us to run a constant temperature simulation.

### Thermostats
In this tutorial we use just NVE (constant number of particles N, volume V and energy E) simulations. However, during the equilibration the kinetic energy is kept constant. While we do a constant energy simulation, it is preferable to give an input temperature, as this is more natural when doing research. Early on there might still be large shifts in kinetic energy due to the equilibration and therefore we keep it constant, using a `VelocityScaling` thermostat. This thermostat simply rescales the velocities to exactly reproduce the input temperature and therefore does not 

## Question 1
Run your first MD simulation with this program. First, make sure conda is activated with `conda activate molsim` and run `pip install .` from the main directory to make sure that the molsim package is correctly installed.

In the cell under this cell you will find the constructor and the run methods of the MD package. Familiarize yourself with the method of setting this up and the different arguments given.

In [ ]:
md = MolecularDynamics(
    numberOfParticles=200,
    temperature=1.0,
    dt=0.005,
    boxSize=8.0,
    numberOfEquilibrationSteps=int(1e4),
    numberOfProductionSteps=int(1e5),
    outputPDB=True,
    logLevel=0,
    seed=12,
    sampleFrequency=10,
)
md.run()

Run this simulation with outputPDB set to `true`. This will generate a file named `movie.pdb` in the current directory. Download this file using

`scp USER@tuva.science.uva.nl:/home/USER/molsim/day3_BasicMD/movie.pdb .`

And visualize with your favourite visualization software (ie. ovito, vmd, ...).

## Question 2
How is one able to control the temperature in this program? After all, the total energy of the system should be constant (not the temperature) and see how the kinetic energy scales with temperature.

In [ ]:
md_T2 = MolecularDynamics(
    numberOfParticles=200,
    temperature=2.0,
    dt=0.005,
    boxSize=8.0,
    numberOfEquilibrationSteps=int(1e4),
    numberOfProductionSteps=int(1e5),
    outputPDB=True,
    logLevel=0,
    seed=12,
    sampleFrequency=100,
)
md_T2.run()

#### Plot the `kineticEnergies` and `observedTemperatures` of two different input temperatures

In [ ]:
fig, ax = plt.subplots(2, figsize=(8, 6))
ax[0].plot(md.time, md.kineticEnergies)
ax[0].plot(md_T2.time, md_T2.kineticEnergies)
ax[1].plot(md.time, md.observedTemperatures)
ax[1].plot(md_T2.time, md_T2.observedTemperatures)
ax[0].set_xlabel(r"time / $\tau$")
ax[0].set_ylabel(r"E$_{kin}$ / $\varepsilon$")

ax[1].set_xlabel(r"time / $\tau$")
ax[1].set_ylabel(r"T / $\varepsilon$")

fig.tight_layout()

## Question 3
To test the energy drift $\Delta E$ of the numerical integration algorithm for a given time step $\Delta t$ after $N$ integration steps, one usually computes

\begin{equation}
\Delta E\left(\Delta t\right) = \frac{1}{N} \sum_{i=1}^{i=N} \left|
\frac{E\left(0\right) - E\left(i\Delta t\right)}{E\left(0\right)}
\right|
\end{equation}

In this equation, $E\left(t\right)$ is the total energy (kinetic+potential) of the system at time $t$. The program prints the average drift at the end of the run. To prevent drift, the timestep should be reasonably small given the system density. High density leads to larger forces, which lead to larger errors. Test to see how high you can push the timestep before the system starts to diverge. 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(md.time, md.potentialEnergies, label="Potential")
ax.plot(md.time, md.kineticEnergies, label="Kinetic")
ax.plot(md.time, md.conservedEnergies, label="Conserved")

ax.set_xlabel(r"Time, t / $\tau$")
ax.set_ylabel(r"Energy, E / $\varepsilon$")
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(md.time, md.conservedEnergies, label="Conserved")

ax.set_xlabel(r"Time, t / $\tau$")
ax.set_ylabel(r"Energy, E / $\varepsilon$")
ax.legend()

## Question 4
An important property of symplectic integrators is that they are time reversible. Can you run the simulation with a negative timestep? Do you expect a difference given the Equations of motion?

In [ ]:
md_rev = MolecularDynamics(
    numberOfParticles=200,
    temperature=1.0,
    dt=-0.005,
    boxSize=8.0,
    numberOfEquilibrationSteps=int(1e4),
    numberOfProductionSteps=int(1e5),
    outputPDB=True,
    logLevel=0,
    seed=12,
    sampleFrequency=1,
)
md_rev.run()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(md_rev.time, md_rev.conservedEnergies, label="Conserved")

ax.set_xlabel(r"Time, t / $\tau$")
ax.set_ylabel(r"Energy, E / $\varepsilon$")
ax.legend()

## Question 5
An important quantity of a liquid or gas is the so-called self diffusivity $D$. There are two methods to calculate $D$:

##### By integrating the velocity autocorrelation function:

\begin{equation}
D = \frac{1}{3} \int_{0}^{\infty} \left\langle \bf{v}(t) \cdot \bf{v} \left(t+t^{'} \right) \right\rangle dt'
\end{equation}

or

\begin{equation}
D = \frac{ \int_{0}^{\infty} \sum_{i=1}^{i=N} \left\langle \bf{v}(i, t) \cdot \bf{v} \left(i, t+t^{'} \right) \right\rangle dt' }{3N}
\end{equation}

in which $N$ is the number of particles and ${\bf v}\left(i,t\right)$
is the velocity of particle $i$ at time $t$. One should choose $t$ in
such a way that independent time origins are taken, \emph{i.e.} $t =
ia\Delta t$, $i=1,2, \cdots , \infty$ and $\left\langle {\bf
v}\left(t\right)\cdot{\bf v}\left(t+a\Delta t\right)\right\rangle
\approx {\bf 0}$ (why?).

##### By calculating the mean square displacement:

\begin{equation}
D = \lim_{t' \to \infty} \frac{\left\langle \left| \bf{x}\left(t+t^{'}\right) - \bf{x}(t) \right|^{2} \right\rangle}{6t'}
\end{equation}

One should be very careful with the mean square displacement calculation when par-
ticles are always translated back to the central box – why?

##### Loading the Mean Square Displacement
The mean square displacement is sampled in `MSDSampler::sample` and can be retrieved by i 


In [ ]:
msd = md.msdSampler.getResults()
fig, ax = plt.subplots(2, 2, figsize=(8, 6))

ax[0, 0].plot(msd[:, 0], msd[:, 1])
ax[1, 0].plot(msd[:, 0], msd[:, 2])
ax[0, 0].set_xlabel(r"Time, t / $\tau$")
ax[0, 0].set_ylabel(r"$\langle |x(t+t') - x(t)|^2 \rangle$")
ax[1, 0].set_xlabel(r"Time, t / $\tau$")
ax[1, 0].set_ylabel(r"$\langle |x(t+t') - x(t)|^2 \rangle / 6t'$")

ax[0, 1].plot(msd[:, 0], msd[:, 3])
ax[1, 1].plot(msd[:, 0], msd[:, 4])
ax[0, 1].set_xlabel(r"Time, t / $\tau$")
ax[0, 1].set_ylabel(r"$\langle v(t) \cdot (t+t') \rangle$")
ax[1, 1].set_xlabel(r"Time, t / $\tau$")
ax[1, 1].set_ylabel(r"$\int \langle v(t) \cdot (t+t') \rangle$")
fig.tight_layout()

# legacy

### Question 2


### Question 3




### Question 4

REPLACE THIS QUESTION WITH ANOTHER PRECOMPUTATION OF (1/X)

As you might have noticed in the code, the implementation of the periodic boundary
conditions is often done as follows:
\begin{equation}
x = x - \text{box}*\text{nint}\left(x*\text{ibox}\right)
\end{equation}
where $\text{ibox}$ is used instead of $1/\text{box}$. Why would one do this?

### Question 5

An important quantity of a liquid or gas is the so-called self diffusivity $D$. There are two methods to calculate $D$:

##### By integrating the velocity autocorrelation function:

\begin{equation}
D = \frac{1}{3} \int_{0}^{\infty} \left\langle \bf{v}(t) \cdot \bf{v} \left(t+t^{'} \right) \right\rangle dt'
\end{equation}

or

\begin{equation}
D = \frac{ \int_{0}^{\infty} \sum_{i=1}^{i=N} \left\langle \bf{v}(i, t) \cdot \bf{v} \left(i, t+t^{'} \right) \right\rangle dt' }{3N}
\end{equation}

in which $N$ is the number of particles and ${\bf v}\left(i,t\right)$
is the velocity of particle $i$ at time $t$. One should choose $t$ in
such a way that independent time origins are taken, \emph{i.e.} $t =
ia\Delta t$, $i=1,2, \cdots , \infty$ and $\left\langle {\bf
v}\left(t\right)\cdot{\bf v}\left(t+a\Delta t\right)\right\rangle
\approx {\bf 0}$ (why?).

##### By calculating the mean square displacement:

\begin{equation}
D = \lim_{t' \to \infty} \frac{\left\langle \left| \bf{x}\left(t+t^{'}\right) - \bf{x}(t) \right|^{2} \right\rangle}{6t'}
\end{equation}

One should be very careful with the mean square displacement calculation when par-
ticles are always translated back to the central box – why?

##### Modify the program such that
the self diffusivity can be
calculated using both methods. Only modifications in subroutine `SampleMSD::sample` are needed. Why is it important to use only independent time origins for the calculation of the mean square displacement and the velocity autocorrelation function? What is the unit of $D$ in SI units? How can one transform $D$ into dimensionless units?


### Question 6

For Lennard-Jones liquids, Naghizadeh and Rice report the
following equation for the self diffusivity (dimensionless units,
$T^* <1.0$ and $p^* <3.0$) 

\begin{equation}
{\rm ^{\rm 10}log}\left(D^* \right) = 0.05 + 0.07p^* - \frac{1.04 +
0.1p^*}{T^*}
\end{equation}

Try to confirm this equation with simulations. 

### Question 7

Instead of calculating the average potential $\left\langle U
\right\rangle$ directly, one can use the radial distribution function
$g\left(r\right)$. Derive an expression for $\left\langle U
\right\rangle$ using $g\left(r\right)$. Compare this calculation with
a direct calculation of the average energy. A similar method can be
used to compute the average pressure.

### Question 8

In the current version of the code, the equation of motion are
integrated by the Verlet algorithm. Make a plot of the energy drift
$\Delta U$ for the following integration algorithms:
- Euler (never use this one except here !!!)
- Verlet
- Velocity Verlet



In [ ]:
rdf = md.rdfSampler.getResults()
msd = md.msdSampler.getResults()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(*rdf.T)
ax.set_xlabel("Distance, r / $\sigma$")
ax.set_ylabel("g(r) / -")

In [ ]:
fig, ax = plt.subplots(2, figsize=(8, 6), sharex=True)
ax[0].plot(msd[:100, 0], msd[:100, 3])
ax[1].plot(msd[:100, 0], msd[:100, 4])

ax[1].set_xlabel("Time")
ax[0].set_ylabel("VACF")
ax[1].set_ylabel("$\int$ d$t$ VACF")
fig.tight_layout()